In [11]:
import os

In [12]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [13]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [14]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config



In [ ]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def Preprocessing(self):
        data = pd.read_csv(self.config.data_path)
        data = data.drop(["STATION","TAVG_ATTRIBUTES","NAME"],axis= 1)

        data['DATE'] = pd.to_datetime(data['DATE'])
        data['YEAR'] = data['DATE'].dt.year
        data['MONTH'] = data['DATE'].dt.month
        data['DAY'] = data['DATE'].dt.day
        data['DAY_OF_WEEK'] = data['DATE'].dt.dayofweek
        data['DAY_OF_YEAR'] = data['DATE'].dt.dayofyear
        data = data.drop('DATE', axis = 1)
        data.to_csv(os.path.join(self.config.root_dir,"Temp_dataset_preprocessed.csv"),index = False)
        logger.info("Preprocessing the data")
        logger.info(data.shape)
        print(data.shape)
    



        




In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.Preprocessing()

except Exception as e:
    raise e

[2025-04-15 14:46:06,435: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 14:46:06,436: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 14:46:06,439: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-15 14:46:06,440: INFO: common: created directory at: artifacts]
[2025-04-15 14:46:06,441: INFO: common: created directory at: artifacts/data_transformation]
[2025-04-15 14:46:06,811: INFO: 1037991296: Splited data into training and test sets]
[2025-04-15 14:46:06,812: INFO: 1037991296: (98446, 9)]
(98446, 9)
